In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.encoders.date import DateEncoder
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

In [2]:
class ReflexiveMemory:
  def __init__(self, reflexSize):
    self.acKey0 = None
    self.pairs = {}
    self.anomalyRM = []
    self.anomalyTM = []
    self.enableLearn = False
    self.tableSize  = reflexSize
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []
    self.entriesDeletedCount = 0
    self.entriesPredictedCount = 0

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data

      table_entries = 0
      oldKey1 = None
      oldKey2 = None
      oldTime = datetime.now()
      for key1, value1 in self.pairs.items():
        table_entries = table_entries + len(value1.items())
        for key2, value2 in value1.items():
          if value2['time'] < oldTime:
            oldKey1 = key1
            oldKey2 = key2
            oldTime = value2['time']
      if table_entries > self.tableSize:
        self.enableLearn = True
        self.entriesDeletedCount = self.entriesDeletedCount + 1
        del self.pairs[oldKey1][oldKey2]
        if len(self.pairs[oldKey1].items()) == 0:
          del self.pairs[oldKey1]

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( denseColumns.dimensions )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr

  def learn(self, denseColumns1, sp, tm):

    tm.activateDendrites(True)
    predictiveColumns = SDR( tm.getPredictiveCells().dimensions[0] )
    predictiveColumns.sparse = list(set(sorted(list(np.where(tm.getPredictiveCells().dense == 1)[0]))))

    activeColumns1 = SDR( predictiveColumns.dimensions )
    sp.compute(denseColumns1, False, activeColumns1)

    denseColumns0 = SDR( denseColumns1.dimensions )
    denseColumns0.sparse = list(map(int, self.acKey0.split('-')))
    activeColumns0 = SDR( predictiveColumns.dimensions )
    sp.compute(denseColumns0, False, activeColumns0)
 
    reflexiveCount, denseReflexiveColumns = self.predict(denseColumns0)
    reflexiveColumns = SDR( predictiveColumns.dimensions )
    if denseReflexiveColumns is not None:
      sp.compute(denseReflexiveColumns, False, reflexiveColumns)

    self.entriesPredictedCount = self.entriesPredictedCount + 1

    overlapRM = 1 - np.count_nonzero((reflexiveColumns.dense & activeColumns1.dense)) / np.count_nonzero(activeColumns1.dense)
    self.anomalyRM.append( overlapRM )

    overlapTM = 1 - np.count_nonzero((predictiveColumns.dense & activeColumns1.dense)) / np.count_nonzero(activeColumns1.dense)
    self.anomalyTM.append( overlapTM )

    self.historyRM.append( reflexiveColumns.dense )
    self.historyTM.append( predictiveColumns.dense )
    self.historyGT.append( activeColumns1.dense )

  def compute(self, denseColumns, sp, tm):
    if self.acKey0 is not None:
      self.learn(denseColumns, sp, tm)  
    self.add(denseColumns)
  

In [3]:
inputSources = [
   "monthly_sp500_pca.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv"
]

In [4]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 2048
}


In [5]:
def anomalyScore(y, x):
    if np.count_nonzero(y) != 0:
        return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
    return 1

def match(y, x, idx1):
  overlap_min = 0.02
  n_samples = len(y)
  score1 = anomalyScore(y[idx1], x[idx1])
  if score1 > (1 - overlap_min):
     idx_closest = None
     score_closest = None
     for idx2 in range(n_samples):
        score2 = anomalyScore(y[idx2], x[idx1])
        if score_closest is None or score_closest > score2:
            score_closest = score2
            idx_closest = idx2
     return idx_closest
  return idx1

def roc_auc_score_multiclass(y_true, y_pred):
    scores = []
    for y_class in set(y_true):
        y_true_class = [True if x == y_class else False for x in y_true]
        y_pred_class = [True if x == y_class else False for x in y_pred]
        scores.append(roc_auc_score(y_true_class, y_pred_class))
    return sum(scores) / len(scores)


In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    tm_infer_tm = 0
    tm_infer_rm = 0

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] )

    inputs = []
   
    try:
        
        for count, record in enumerate(records):

            consumption = float(record[1])
            inputs.append( consumption )
            consumptionBits = scalarEncoder.encode(consumption)
            encoding = SDR( consumptionBits )
            
            activeColumns = SDR( sp.getColumnDimensions() )

            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            rm.compute(encoding, sp, tm)

            sp.compute(encoding, learn_sp, activeColumns)

            tmp_tm = time.time()
            tm.compute(activeColumns, learn=learn_tm)
            tm_infer_tm = tm_infer_tm + (time.time() - tmp_tm)

            tmp_tm = time.time()
            rm.predict(encoding)
            tm_infer_rm = tm_infer_rm + (time.time() - tmp_tm)

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    metric = {}
    metric['dataset'] = dataset
    metric['rm-delete-count'] = rm.entriesDeletedCount
    metric['rm-predict-count'] = rm.entriesPredictedCount
    metric['inputt-count'] = len(inputs)

    metric['key0-count'] = len(rm.pairs)
    key1_max_value = -1
    for key1, value1 in rm.pairs.items():
        for key2, value2 in value1.items():
            if key1_max_value < value2["count"]:
                key1_max_value = value2["count"]
    metric['key1-max-value'] = key1_max_value
    metric['key-pair-max-len'] = max([ len(val0) for val0 in rm.pairs.values()])
    metric['key-pair-counts'] = str([ len(val0) for val0 in rm.pairs.values()])

    dataset_metrics.append(metric)
    pbar.update(1)

pbar.close()

100%|██████████| 14/14 [00:37<00:00,  2.68s/it]


In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)
df

,dataset,rm-delete-count,rm-predict-count,inputt-count,key0-count,key1-max-value,key-pair-max-len,key-pair-counts
0,monthly_sp500_pca.csv,0,1641,1642,769,4,11,"[6, 8, 8, 9, 8, 8, 8, 2, 5, 5, 6, 6, 4, 4, 6, ..."
1,weekly_dow_jones.csv,27,2079,2080,1823,2,4,"[3, 3, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, ..."
2,weekly_nasdaq.csv,0,2080,2081,1469,3,7,"[4, 1, 3, 1, 1, 2, 4, 3, 3, 3, 1, 1, 1, 1, 3, ..."
3,weekly_sp500.csv,0,2081,2082,1161,4,7,"[6, 4, 3, 3, 2, 5, 2, 4, 2, 2, 4, 6, 7, 6, 4, ..."
4,monthly_vix_close.csv,0,4050,4051,72,224,14,"[12, 8, 12, 10, 10, 13, 10, 13, 10, 9, 7, 8, 9..."
5,monthly_vix_high.csv,0,4050,4051,82,192,15,"[10, 11, 9, 8, 10, 11, 10, 11, 12, 12, 8, 8, 9..."
6,monthly_vix_low.csv,0,4050,4051,70,241,13,"[8, 11, 9, 8, 8, 10, 9, 9, 9, 7, 7, 5, 4, 10, ..."
7,monthly_vix_open.csv,0,4050,4051,75,210,13,"[11, 9, 8, 11, 8, 9, 11, 10, 11, 9, 9, 6, 4, 1..."
8,daily_natural_gas.csv,0,5798,5799,18,1392,6,"[5, 6, 4, 3, 2, 6, 5, 4, 6, 5, 4, 5, 1, 5, 5, ..."
9,daily_oil_prices.csv,0,8300,8301,151,372,11,"[5, 5, 6, 5, 3, 4, 3, 3, 4, 4, 4, 3, 4, 6, 5, ..."
